In [ ]:
# Cellule 1 - Imports
import pandas as pd
import numpy as np
from statsmodels.api import GLM
from statsmodels.families import Poisson, Tweedie
import matplotlib.pyplot as plt

In [ ]:
# Cellule 2 - Fonctions
def load_data():
    # Chargement des données d'entraînement
    train_df = pd.read_csv('data/train_output_DzPxaPY.csv')
    
    # Chargement des données de test
    test_df = pd.read_csv('data/test_input_5qJzHrr.csv')
    
    # Chargement du format de soumission
    submission_example = pd.read_csv('data/submission_csv_file_random_example_3fbDtrr (1).csv')
    
    return train_df, test_df, submission_example

def prepare_data(train_df, test_df):
    # Séparation des features et targets dans le train
    y_freq = train_df['FREQ']
    y_cost = train_df['CM']
    X_train = train_df.drop(['FREQ', 'CM', 'CHARGE', 'ID'], axis=1)
    
    # Préparation du test
    X_test = test_df.drop(['ID'], axis=1)
    
    # Encodage des variables catégorielles
    categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']
    
    # Création d'un DataFrame combiné pour l'encodage
    combined_data = pd.concat([X_train, X_test], axis=0)
    encoded_data = pd.get_dummies(combined_data, columns=categorical_cols)
    
    # Séparation des données encodées
    X_train_encoded = encoded_data[:len(X_train)]
    X_test_encoded = encoded_data[len(X_train):]
    
    return X_train_encoded, X_test_encoded, y_freq, y_cost

def train_models(X_train, y_freq, y_cost):
    # Modèle de fréquence
    freq_model = GLM(
        y_freq,
        X_train,
        family=Poisson(link='log')
    ).fit()
    
    # Modèle de coût
    cost_model = GLM(
        y_cost,
        X_train,
        family=Tweedie(link='log', var_power=1.5)
    ).fit()
    
    return freq_model, cost_model

def generate_submission(freq_model, cost_model, X_test, test_df, submission_example):
    # Prédictions
    pred_freq = freq_model.predict(X_test)
    pred_cost = cost_model.predict(X_test)
    
    # Calcul de la charge prédite
    pred_charge = pred_freq * pred_cost * test_df['ANNEE_ASSURANCE']
    
    # Création du fichier de soumission
    submission = pd.DataFrame({
        'ID': test_df['ID'],
        'CHARGE': pred_charge
    })
    
    # Vérification du format avec l'exemple
    assert all(submission.columns == submission_example.columns), "Format de colonnes incorrect"
    
    return submission

In [ ]:
# Cellule 3 - Exécution du pipeline
# Chargement des données
print("Chargement des données...")
train_df, test_df, submission_example = load_data()

# Préparation des données
print("Préparation des données...")
X_train, X_test, y_freq, y_cost = prepare_data(train_df, test_df)

# Entraînement des modèles
print("Entraînement des modèles...")
freq_model, cost_model = train_models(X_train, y_freq, y_cost)

# Génération de la soumission
print("Génération du fichier de soumission...")
submission = generate_submission(freq_model, cost_model, X_test, test_df, submission_example)

# Sauvegarde du fichier de soumission
submission.to_csv('submission.csv', index=False)
print("Fichier de soumission généré avec succès!")

In [ ]:
# Cellule 4 - Visualisation des résultats
# Afficher quelques statistiques sur les prédictions
print("Statistiques des prédictions :")
print(submission['CHARGE'].describe())

# Visualisation de la distribution des prédictions
plt.figure(figsize=(10, 6))
plt.hist(submission['CHARGE'], bins=50)
plt.title('Distribution des charges prédites')
plt.xlabel('Charge')
plt.ylabel('Fréquence')
plt.show()